In [ ]:
import re
import pandas as pd
import numpy as np

!pip install tqdm
from tqdm import tqdm
# use tqdm with pandas
# use progress_apply instead of normal apply
tqdm.pandas()

import string
!pip install plotly
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

# NLTK STUFF
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.snowball import EnglishStemmer
from nltk import word_tokenize

/opt/venv/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Let's import the data

In [ ]:
sample_submission = pd.read_csv("/datasets/real-or-not-dataset/sample_submission.csv")
test = pd.read_csv("/datasets/real-or-not-dataset/test.csv")
train = pd.read_csv("/datasets/real-or-not-dataset/train.csv")

In [ ]:
n = 10
random_state = 3
modified = train
modded_test = test

In [ ]:
stop_w = stopwords.words('english')
stemmer = EnglishStemmer(ignore_stopwords=True)
!pip install flashgeotext
# for finding the city names properly
from flashgeotext.geotext import GeoText
geotext = GeoText(use_demo_data=True)

2020-06-26 11:40:14.152 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-06-26 11:40:14.157 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-06-26 11:40:14.158 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


Process the Cols

In [ ]:
# Process non null keywords and locations
modified["keyword"].fillna("", inplace = True)
modified["location"].fillna("", inplace = True)
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,Inexpressible Island,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,Los Angeles,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,Purfleet,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,Nigeria,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,"Afghanistan, USA",#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,Detroit,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,"New Delhi,India",Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body%20bag,New York,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,"Atlanta, GA",Am I hearing thunder or trucks?,0


In [ ]:
def makecountryarray(x):
    countries = geotext.extract(input_text=str(x), span_info=False)['countries']
    country = ""
    for items in countries:
        country =  country + ' ' + items
    # print("location : {}, got {}".format(x, country))
    return country
print(makecountryarray("I'd like to build a wall between Mexico and Usa"))

 Mexico United States


In [ ]:
# Convert the weird location to a more standardized tag
modified.location = modified.location.progress_apply(makecountryarray)

100%|██████████| 7613/7613 [00:00<00:00, 102437.94it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,Afghanistan United States,#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,India,Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body%20bag,,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,,Am I hearing thunder or trucks?,0


In [ ]:
def vectandstem(x):
    tokenized = word_tokenize(str(x))
    
    full= " ".join(tokenized)
    return full

print(vectandstem("emergencies totally"))

emergencies totally


In [ ]:
# Vectorize and stem the keywords
modified.keyword = modified.keyword.progress_apply(vectandstem)

100%|██████████| 7613/7613 [00:00<00:00, 11388.20it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,The Catastrophic Effects of Hiroshima and Naga...,1
5763,8225,riot,,@eac4AU You can now PRE-ORDER the film on ITUN...,0
5157,7356,obliterate,,Whereas Jez will obliterate the national debt ...,1
4550,6467,injured,,Washington Post - 4 dead dozens injured in Gaz...,1
4329,6148,hijack,,Criminals Who Hijack Lorries And Buses Arreste...,1
1420,2049,casualties,Afghanistan United States,#Afghanistan: sharp rise in women and children...,1
6984,10016,twister,,Crazy Mom Threw Teen Daughter a NUDE Twister S...,0
7541,10782,wreckage,India,Wreckage 'Conclusively Confirmed' as From MH37...,1
979,1418,body % 20bag,,genuine Leather man Bag Messenger fit iPad min...,0
6671,9562,thunder,,Am I hearing thunder or trucks?,0


In [ ]:
#Function to Vectorize and clean text
def hiICleanTrash(x):
    # first pass to clean the punctuation
    pass1  = "".join([char for char in x if char not in string.punctuation])
    # second pass to clean the numbers up
    pass2 = re.sub('[0-9]+', '', pass1)
    # third pass to tokenize
    pass3 = word_tokenize(pass2)
    # fourth pass to use snowball to stem words if they're not a stopword
    pass4 = [word for word in pass3 if word not in stop_w]
    return (pass4)
print(hiICleanTrash("Hello! I like cleaning the trash. I found 4 occurances today!"))

['Hello', 'I', 'like', 'cleaning', 'trash', 'I', 'found', 'occurances', 'today']


In [ ]:
# print(set(stop_w))

In [ ]:
modified.text = modified.text.progress_apply(hiICleanTrash)
modded_test.text = modded_test.text.progress_apply(hiICleanTrash)

100%|██████████| 3263/3263 [00:00<00:00, 4989.87it/s]


In [ ]:
modified.sample(n=n,random_state=random_state)

,id,keyword,location,text,target
1506,2174,catastrophic,,"[The, Catastrophic, Effects, Hiroshima, Nagasa...",1
5763,8225,riot,,"[eacAU, You, PREORDER, film, ITUNES, amp, watc...",0
5157,7356,obliterate,,"[Whereas, Jez, obliterate, national, debt, giv...",1
4550,6467,injured,,"[Washington, Post, dead, dozens, injured, Gaza...",1
4329,6148,hijack,,"[Criminals, Who, Hijack, Lorries, And, Buses, ...",1
1420,2049,casualties,Afghanistan United States,"[Afghanistan, sharp, rise, women, children, ca...",1
6984,10016,twister,,"[Crazy, Mom, Threw, Teen, Daughter, NUDE, Twis...",0
7541,10782,wreckage,India,"[Wreckage, Conclusively, Confirmed, From, MH, ...",1
979,1418,body % 20bag,,"[genuine, Leather, man, Bag, Messenger, fit, i...",0
6671,9562,thunder,,"[Am, I, hearing, thunder, trucks]",0


In [ ]:
modded_test.sample(n=n,random_state=random_state)

,id,keyword,location,text
805,2646,crashed,USA,"[Website, Malfunctioning, PHP, Scripts, workin..."
2259,7511,oil%20spill,Houston,"[California, Spring, Oil, Spill, Estimate, Gro..."
3091,10233,volcano,"Hawaii, USA","[USGS, M, km, S, Volcano, Hawaii, Time, UTC, e..."
1584,5360,fire,Wonderland,"[When, likes, Fire, IG, selfie]"
2941,9742,tragedy,NaN,"[Rly, tragedy, MP, Some, live, recount, horror..."
1177,3882,detonation,ShloMotion,"[JerusalemPost, WATCH, Israel, performs, contr..."
2235,7450,obliteration,Canada,"[Need, conquest, server, CTE, theyre, Russian,..."
1893,6378,hostages,Rocky Mountains,"[Sinjar, Massacre, Yazidis, Blast, Lack, Actio..."
3228,10728,wreck,Philippians 4:13,"[Dj, wreck, cut, beat]"
1141,3767,destruction,Hell,"[iamdestruction, Okay, Ill, put, tail, first, ..."


In [ ]:
setofwords = set(modified["text"].sum())
print(len(setofwords))

25832


In [ ]:
# Create a new df
model_df = pd.DataFrame()

In [ ]:
def get_info():
    fig = px.box(model_df.melt(var_name="Model"),
        y="value",labels={
            'value':'Accuracy'
        },
        facet_col="Model", color="Model",
        boxmode="overlay",
        points='all')
    # Remove the Model = **
    fig.for_each_annotation(lambda a: a.update(text=''))
    fig.show()
    

Getting started for using tensorflow and keras

In [ ]:
# Tensorflow and Keras imports
import tensorflow as tf
# Importing the layers we may use
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
# keras preprocessing tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Hyperparameters
vocab_size = 10000
embedding_dim = 32
# idk for max length, tweets are like 280chars long
max_length = 80
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
dropout_ratio = 0.5
training_portion = .8
# tf stuff
print(tf.__version__)
print(tf.executing_eagerly())
test_corpus=modded_test['text']
#input_dim: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
#output_dim: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.
#input_length: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.

2.2.0
True


## Run till before the hiIcleanTrash cell!!!

In [ ]:
def make_splits(modified, verbose=0):
    train_size = int(len(modified) * training_portion)
    # train split 
    train_corpus = modified['text'][0: train_size]
    train_labels = modified['target'][0: train_size]
    # validation split
    validation_corpus = modified['text'][train_size:]
    validation_labels = modified['target'][train_size:]
    if(verbose):
        print(train_size)
        print((train_mat))
        print((train_labels))
        print((validation_mat))
        print((validation_labels))
    return train_corpus, train_labels, validation_corpus, validation_labels

In [ ]:
train_corpus, train_labels, validation_corpus, validation_labels = make_splits(modified)

In [ ]:
test

,id,keyword,location,text
0,0,NaN,NaN,"[Just, happened, terrible, car, crash]"
1,2,NaN,NaN,"[Heard, earthquake, different, cities, stay, s..."
2,3,NaN,NaN,"[forest, fire, spot, pond, geese, fleeing, acr..."
3,9,NaN,NaN,"[Apocalypse, lighting, Spokane, wildfires]"
4,11,NaN,NaN,"[Typhoon, Soudelor, kills, China, Taiwan]"
5,12,NaN,NaN,"[Were, shakingIts, earthquake]"
6,21,NaN,NaN,"[Theyd, probably, still, show, life, Arsenal, ..."
7,22,NaN,NaN,"[Hey, How]"
8,27,NaN,NaN,"[What, nice, hat]"
9,29,NaN,NaN,[Fuck]


In [ ]:
def make_tokenized_sequences(train_corpus, validation_corpus,test_corpus, verbose=0):
    tokenizer = Tokenizer(num_words=vocab_size,
                          oov_token=oov_tok,
                          filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                          lower=True)
    tokenizer.fit_on_texts(train_corpus)

    word_index = tokenizer.word_index
    if(verbose):
        dict(list(word_index.items())[0:10])
    
    train_sequences = tokenizer.texts_to_sequences(train_corpus)
    train_padded = pad_sequences(train_sequences, maxlen=max_length,
                                    padding=padding_type, truncating=trunc_type)

    validation_sequences = tokenizer.texts_to_sequences(validation_corpus)
    validation_padded = pad_sequences(validation_sequences, maxlen=max_length,
                                        padding=padding_type, truncating=trunc_type)

    test_sequences = tokenizer.texts_to_sequences(test_corpus)
    test_padded = pad_sequences(test_sequences, maxlen=max_length,
                                        padding=padding_type, truncating=trunc_type)
    
    if(verbose):
        print(train_sequences[10])
        print(train_mat[10])
    
    return train_padded, validation_padded, test_padded

In [ ]:
train_padded, validation_padded, test_padded = make_tokenized_sequences(train_corpus, validation_corpus, test_corpus)

In [ ]:
def compile_and_run(num_epochs, model_name='best_model', verbose=1, dropout_ratio=dropout_ratio):
    model = Sequential([
    # add an Embedding layer expecting input vocab of size vocab_size,
    # and output embedding dimension of size embedding_dim
    Embedding(vocab_size, embedding_dim),
    Bidirectional(LSTM(embedding_dim)),
    Dropout(dropout_ratio),
    Dense(embedding_dim,activation='relu'),
    Dense(2, activation='softmax')
    ])
    if(verbose):
        model.summary()
        print("Dropout ratio is {}".format(dropout_ratio))
    
    # Early Stopping callback setup
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.05,
                                        mode='min', patience=10, verbose=1)

    # Model Checkpoint callback setup
    mc = tf.keras.callbacks.ModelCheckpoint(model_name+'.h5', monitor='val_accuracy',
                                            mode='max', verbose=1, save_best_only=True)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

    history = model.fit(train_padded, train_labels,epochs=num_epochs, validation_data=(validation_padded, validation_labels), verbose=verbose, callbacks=[es, mc], shuffle=True)

    return history

model_history = compile_and_run(20, dropout_ratio=0.5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,181,102
Trainable params: 1,181,102
Non-trainable params: 0
_________________________________________________________________
Dropout ratio is 0.5
Epoch 1/20
188/191 [============================>.] - ETA: 0s - loss: 0.5410 - accuracy: 0.

In [ ]:
def eval_best_model(model_name='best_model'):
    saved_model = tf.keras.models.load_model(model_name+'.h5')
    saved_model.summary()
    # evaluate the model
    _, train_acc = saved_model.evaluate(train_padded, train_labels, verbose=0)
    _, test_acc = saved_model.evaluate(validation_padded, validation_labels, verbose=0)

    print('Train: %.5f, Test: %.5f' % (train_acc, test_acc))

eval_best_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 1,181,102
Trainable params: 1,181,102
Non-trainable params: 0
_________________________________________________________________
Train: 0.87718, Test: 0.77741


Doing the submission steps:

In [ ]:
saved_model = tf.keras.models.load_model('best_model.h5')

predicted  = saved_model.predict(test_padded)

In [ ]:
sub=pd.DataFrame({'id':test['id'].values.tolist(),'target':tf.argmax(input=predicted, axis=1)})
sub.sample(n=n,random_state=random_state)

NameError: name 'pd' is not defined

In [ ]:
sub.to_csv('submission.csv',index=False)

In [1]:
## NEW MODELS

In [ ]:
#new models 

#GloVe is an unsupervised learning algorithm
# for obtaining vector representations for words
zip_file = "http://nlp.stanford.edu/data/glove.6B.zip"
import requests, zipfile, io
r = requests.get(zip_file)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
#print("no of words in glove dictionary is "+str(len(embeddings_index)))
#IGNORE
embedding_matrix = np.zeros((VOCAB_SIZE, EMBED_DIM))
index  = 0
def embedpls(word,index):
    for element in word:
        embedding_vector = embeddings_index.get(element)
        print(index)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
        index=index+1  
modified["text"].apply(lambda x: embedpls(x,index))

2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
0
1
2
3
4
5
6
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4

0       None
1       None
2       None
3       None
4       None
        ... 
7608    None
7609    None
7610    None
7611    None
7612    None
Name: text, Length: 7613, dtype: object

In [ ]:
print(embedding_matrix[31])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [ ]:
texts = [text for text in modified["text"]]
tokenizer = Tokenizer(filters= "'!#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n") 
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

In [ ]:
#print(word_index)  #word,index  it is a dictionary

AttributeError: 'dict' object has no attribute 'iloc'

In [ ]:
VOCAB_SIZE = len(word_index)
EMBED_DIM = 100

In [ ]:
embedding_matrix = np.zeros((VOCAB_SIZE, EMBED_DIM)) #has the weights
for word,index in word_index.items():
    if embeddings_index.get(word) is not None: #get() returns a value for the given key
        embedding_matrix[index] = embeddings_index.get(word) #has the coeff of the weight 

In [ ]:
embedding_matrix.shape
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.046539    0.61966002  0.56647003 ... -0.37616    -0.032502
   0.80620003]
 [-0.038194   -0.24487001  0.72812003 ... -0.1459      0.82779998
   0.27061999]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [ ]:
def glove(num_epochs, model_name='glove_model', verbose=1, dropout_ratio=dropout_ratio):
    model = Sequential([
    # add an Embedding layer expecting input vocab of size vocab_size,
    # and output embedding dimension of size embedding_dim
    Embedding(VOCAB_SIZE,EMBED_DIM,weights=[embedding_matrix],trainable=False),
    Bidirectional(LSTM(EMBED_DIM)),
    Dropout(dropout_ratio),
    Dense(EMBED_DIM,activation='relu'),
    Dense(2, activation='softmax')
    ])
    if(verbose):
        model.summary()
        print("Dropout ratio is {}".format(dropout_ratio))
    
    # Early Stopping callback setup
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.05,
                                        mode='min', patience=10, verbose=1)

    # Model Checkpoint callback setup
    mc = tf.keras.callbacks.ModelCheckpoint(model_name+'.h5', monitor='val_accuracy',
                                            mode='max', verbose=1, save_best_only=True)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

    history = model.fit(train_padded, train_labels,epochs=num_epochs, validation_data=(validation_padded, validation_labels), verbose=verbose, callbacks=[es, mc], shuffle=True)

    return history

model_history = glove(20, dropout_ratio=0.3)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         2181000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 2,362,102
Trainable params: 181,102
Non-trainable params: 2,181,000
_________________________________________________________________
Dropout ratio is 0.3
Epoch 1/20
188/191 [============================>.] - ETA: 0s - loss: 0.6576 - accu